# Purpose
Facilitate discussion of potential to optimize curve fitting via the lmfit library.

# Simplifications
The model in this document is a simplified version of the actual model.  Here, it is a three exponential decay with well separated decay time scales and a fixed amplitude at x = 0.  We also have three separate multiexponentials that need to be solved together.  For simplicity, the parameters for each multiexponential are the same value.  This simplification is not the case in reality.  Additionally, this notebook also does not have fully optimized model computation, but it is sufficient for the discussion.

# Conclusions
In this notebook, we can see that `getval` accounts for a significant portion of cumulative time when one of the parameters is defined using `expr`.  If the constraint defined using `expr` is removed, the total computation time increases and the percent cumulative time decreases.  The model can be rewritten to explicitly include the constraint in the model (this reduces the flexibility, but will be fine for this particular task) which provides a lower percent contribution of `getval` and lower overall computation time.

#### Process with Constrained Amplitude (using expr in parameters)
|Process Name|Cumulative Time (s)|Percent of Multiexponential|
|-|-|-|
|general_multiexponential|7.1|100|
|compute_multiexponentials|2.6|37|
|_getval|3.9|55|


#### Process without Constrained Amplitude
|Process Name|Cumulative Time (s)|Percent of Multiexponential|
|-|-|-|
|general_multiexponential|15.9|100|
|compute_multiexponentials|9.9|62|
|_getval|2.0|13|



#### Process with Constrained Amplitude (coded into model, no expr in parameters)
|Process Name|Cumulative Time (s)|Percent of Multiexponential|
|-|-|-|
|hardcoded_multiexponential|3.1|100|
|compute_multiexponentials|2.0|65|
|_getval|0.3|10|

The effects of using `expr` in a Parameters object is further investigated with the conclusion that `param[name]` is 10x faster than `param[name].value` when using `value` instead of `expr`.  When `expr` is used, the difference becomes 100x.

In [1]:
import lmfit
import numba
import numpy as np

from sys import version

In [2]:
@numba.njit()
def compute_exponential(x, A, T):
    return A * np.exp(-1 * x / T)

@numba.njit()
def compute_multiexponentials(x, As, Ts):
    y = x * 0
    for A, T in zip(As, Ts):
        y += compute_exponential(x, A, T)
    return y

def general_multiexponential(params, x, data=0, sigma=1):
    for i in range(1, 4):
        y = []
        As = []
        Ts = []
        for j in range(1, 4):
            A = params[f'A_{i}_{j}'].value
            T = params[f'T_{i}_{j}'].value
            As.append(A)
            Ts.append(T)
        yi = compute_multiexponentials(x, As, Ts)
        y.append(yi)
        
    y = np.array(y).flatten()
    return (y - data) / sigma

def hardcoded_multiexponential(params, x, data=0, sigma=1):
    for i in range(1, 4):
        y = []
        As = []
        Ts = []
        for j in range(1, 4):
            if j == 3:
                A = 1 - As[0] - As[1]
            else:
                A = params[f'A_{i}_{j}'].value
            T = params[f'T_{i}_{j}'].value
            As.append(A)
            Ts.append(T)
        yi = compute_multiexponentials(x, As, Ts)
        y.append(yi)
        
    y = np.array(y).flatten()
    return (y - data) / sigma

In [3]:
error = 0.01
np.random.seed(42)

In [4]:
params_1 = lmfit.Parameters()
for i in range(1, 4):
    params_1.add(f'A_{i}_1', value=0.3, min=0, max=1)
    params_1.add(f'A_{i}_2', value=0.3, min=0, max=1)
    params_1.add(f'A_{i}_3', expr=f'1 - A_{i}_1 - A_{i}_2')
    params_1.add(f'T_{i}_1', value=1, min=0.5, max=2)
    params_1.add(f'T_{i}_2', value=10, min=5, max=20)
    params_1.add(f'T_{i}_3', value=100, min=50, max=200)
    
x = np.concatenate([np.array([0]), np.logspace(-3, 3, 99)])
data = general_multiexponential(params_1, x) + error * np.random.randn(len(x))
minimizer_1 = lmfit.Minimizer(general_multiexponential, params_1, fcn_args=(data, error))

In [5]:
%%prun -l 20
result_1 = minimizer_1.minimize(method='least_squares')

         5386025 function calls (5100928 primitive calls) in 11.698 seconds

   Ordered by: internal time
   List reduced from 1960 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
264894/37842    1.318    0.000    2.505    0.000 asteval.py:279(run)
     6305    0.834    0.000    7.145    0.001 <ipython-input-2-4eb2342400e6>:12(general_multiexponential)
     1482    0.797    0.001    0.798    0.001 ffi.py:106(__call__)
   132477    0.720    0.000    3.884    0.000 parameter.py:745(_getval)
    18915    0.700    0.000    2.584    0.000 <ipython-input-2-4eb2342400e6>:5(compute_multiexponentials)
   151364    0.346    0.000    1.903    0.000 typeof.py:24(typeof)
   867571    0.297    0.000    0.301    0.000 {built-in method builtins.isinstance}
     6305    0.294    0.000   10.355    0.002 minimizer.py:485(__residual)
75684/37842    0.287    0.000    1.583    0.000 asteval.py:581(on_binop)
   151322    0.284    0.000    0.606    0.000 

In [6]:
params_2 = lmfit.Parameters()
for i in range(1, 4):
    params_2.add(f'A_{i}_1', value=0.3, min=0, max=1)
    params_2.add(f'A_{i}_2', value=0.3, min=0, max=1)
    params_2.add(f'A_{i}_3', value=0.4, min=0, max=1)
    params_2.add(f'T_{i}_1', value=1, min=0.5, max=2)
    params_2.add(f'T_{i}_2', value=10, min=5, max=20)
    params_2.add(f'T_{i}_3', value=100, min=50, max=200)
minimizer_2 = lmfit.Minimizer(general_multiexponential, params_2, fcn_args=(data, error))

In [7]:
%%prun -l 20
result_2 = minimizer_2.minimize(method='least_squares')

         11740925 function calls in 25.887 seconds

   Ordered by: internal time
   List reduced from 192 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    27000    3.015    0.000   15.881    0.001 <ipython-input-2-4eb2342400e6>:12(general_multiexponential)
    81000    2.455    0.000    9.890    0.000 <ipython-input-2-4eb2342400e6>:5(compute_multiexponentials)
   486072    1.806    0.000    1.981    0.000 parameter.py:745(_getval)
   648000    1.325    0.000    7.436    0.000 typeof.py:24(typeof)
    27000    1.212    0.000   20.999    0.001 minimizer.py:485(__residual)
   648000    1.113    0.000    2.355    0.000 numpy_support.py:154(map_arrayscalar_type)
   648000    1.024    0.000    5.277    0.000 functools.py:819(wrapper)
   648000    0.854    0.000    1.076    0.000 numpy_support.py:85(from_dtype)
   648000    0.676    0.000    1.268    0.000 functools.py:765(dispatch)
   486000    0.670    0.000    0.816    0.000 paramet

In [8]:
params_3 = lmfit.Parameters()
for i in range(1, 4):
    params_3.add(f'A_{i}_1', value=0.3, min=0, max=1)
    params_3.add(f'A_{i}_2', value=0.3, min=0, max=1)
    params_3.add(f'T_{i}_1', value=1, min=0.5, max=2)
    params_3.add(f'T_{i}_2', value=10, min=5, max=20)
    params_3.add(f'T_{i}_3', value=100, min=50, max=200)
minimizer_3 = lmfit.Minimizer(hardcoded_multiexponential, params_3, fcn_args=(data, error))

In [9]:
%%prun -l 20
result_3 = minimizer_3.minimize(method='least_squares')

         2669057 function calls in 5.174 seconds

   Ordered by: internal time
   List reduced from 192 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     6305    0.588    0.000    3.127    0.000 <ipython-input-2-4eb2342400e6>:28(hardcoded_multiexponential)
    18915    0.474    0.000    2.006    0.000 <ipython-input-2-4eb2342400e6>:5(compute_multiexponentials)
    94635    0.312    0.000    0.344    0.000 parameter.py:745(_getval)
   151320    0.273    0.000    1.532    0.000 typeof.py:24(typeof)
   151320    0.232    0.000    0.492    0.000 numpy_support.py:154(map_arrayscalar_type)
     6305    0.221    0.000    4.089    0.001 minimizer.py:485(__residual)
   151320    0.213    0.000    1.088    0.000 functools.py:819(wrapper)
   151320    0.177    0.000    0.224    0.000 numpy_support.py:85(from_dtype)
   151320    0.138    0.000    0.256    0.000 functools.py:765(dispatch)
   151320    0.127    0.000    0.619    0.000 typeof.

In [11]:
fast_params = lmfit.Parameters()
fast_params.add('fast', value=2)

%timeit fast_params['fast']
%timeit fast_params['fast'].value

180 ns ± 8.2 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
2.43 µs ± 323 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [12]:
slow_params = lmfit.Parameters()
slow_params.add('slow', expr='2')

%timeit slow_params['slow']
%timeit slow_params['slow'].value

167 ns ± 15.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
17.6 µs ± 2.42 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [13]:
both_params = lmfit.Parameters()
both_params.add('slow', expr='2')
both_params.add('fast', value=2)

%timeit both_params['slow']
%timeit both_params['slow'].value
%timeit both_params['fast']
%timeit both_params['fast'].value

194 ns ± 17.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
17 µs ± 3.08 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
207 ns ± 33.1 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
1.99 µs ± 128 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [14]:
print(version)
print(f'lmfit version: {lmfit.__version__}')
print(f'numba version: {numba.__version__}')
print(f'numpy version: {np.__version__}')

3.7.1 | packaged by conda-forge | (default, Nov 13 2018, 19:01:41) [MSC v.1900 64 bit (AMD64)]
lmfit version: 0.9.13
numba version: 0.42.0
numpy version: 1.15.4
